In [1]:
# install mysql connector python if not available
# pip install pandas mysql-connector-python

In [2]:
import pandas as pd
import mysql.connector
import warnings
import json
from datetime import datetime
warnings.filterwarnings("ignore")

In [3]:
# Replace placeholders with your MySQL credentials
host = '127.0.0.1'

# user account having access to database
user = '*************'

# authentication password for the user
password = '***********'
database_name = 'northwind'

# Establish a connection to the MySQL database
try:
    connection = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database_name
    )
    if connection.is_connected():
        print("Connected to MySQL database")

except Exception as e:
    print("Error:", str(e))

Connected to MySQL database


In [4]:
# Query to retrieve all the tables in the database
query = "SHOW TABLES"

# Fetch the data into a DataFrame
try:
    df = pd.read_sql(query, con=connection)
    print(f"Fetched {len(df)} tables from {database_name} database")

except Exception as e:
    print("Error:", str(e))

Fetched 8 tables from northwind database


In [5]:
df

,Tables_in_northwind
0,categories
1,customers
2,employees
3,orderdetails
4,orders
5,products
6,shippers
7,suppliers


In [6]:
# creating query column to fetch all the data from each table
df['queries'] = "SELECT * FROM " + df['Tables_in_northwind']
df

,Tables_in_northwind,queries
0,categories,SELECT * FROM categories
1,customers,SELECT * FROM customers
2,employees,SELECT * FROM employees
3,orderdetails,SELECT * FROM orderdetails
4,orders,SELECT * FROM orders
5,products,SELECT * FROM products
6,shippers,SELECT * FROM shippers
7,suppliers,SELECT * FROM suppliers


# save entire database in json format

In [7]:
# define an empty list, to further append tables data
Tables = []
for i in range(8):
    globals()[df['Tables_in_northwind'][i]] = pd.read_sql(df['queries'][i], con=connection)
    Tables.append(globals()[df['Tables_in_northwind'][i]])
print('Following tables were created and are grouped as list: Tables\n', list(df['Tables_in_northwind']))

Following tables were created and are grouped as list: Tables
 ['categories', 'customers', 'employees', 'orderdetails', 'orders', 'products', 'shippers', 'suppliers']


In [8]:
# for parsing dataframes as json, datetime columns should be converted to string type
orders['OrderDate'] = orders['OrderDate'].astype(str)
employees['BirthDate'] = employees['BirthDate'].astype(str)

In [9]:
#create and save json data to a file name northwind.json
# Specify the file path where you want to save the JSON data
file_path = r"C:\Users\saian\Documents\northwind.json"

# Convert DataFrames to dictionaries
Tables_dict_list = [table.to_dict(orient='records') for table in Tables]

# Using a loop to create: dictionary with table names
Tables_json = {}
for i in range(8):
    Tables_json[df['Tables_in_northwind'][i]] = Tables_dict_list[i]
    
    
# Write the list of dictionaries to the JSON file
with open(file_path, "w") as json_file:
    json.dump(Tables_json, json_file)
    
print(f"List has been written to {file_path}")

List has been written to C:\Users\saian\Documents\northwind.json


# Read the json file back and create tables

In [10]:
with open('northwind.json', 'r') as json_file:
    data = json.load(json_file)

In [11]:
# Iterate over dictionary keys and values
for key in data.keys():
    data[key] = pd.DataFrame(data[key])

In [12]:
# Create variables from dictionary keys using globals()
globals().update(data)

In [13]:
categories

,CategoryID,CategoryName,Description
0,1,Beverages,"Soft drinks, coffees, teas, beers, and ales"
1,2,Condiments,"Sweet and savory sauces, relishes, spreads, an..."
2,3,Confections,"Desserts, candies, and sweet breads"
3,4,Dairy Products,Cheeses
4,5,Grains/Cereals,"Breads, crackers, pasta, and cereal"
5,6,Meat/Poultry,Prepared meats
6,7,Produce,Dried fruit and bean curd
7,8,Seafood,Seaweed and fish


In [14]:
employees.head()

,EmployeeID,LastName,FirstName,BirthDate,Photo,Notes
0,1,Davolio,Nancy,1968-12-08,EmpID1.pic,Education includes a BA in psychology from Col...
1,2,Fuller,Andrew,1952-02-19,EmpID2.pic,Andrew received his BTS commercial and a Ph.D....
2,3,Leverling,Janet,1963-08-30,EmpID3.pic,Janet has a BS degree in chemistry from Boston...
3,4,Peacock,Margaret,1958-09-19,EmpID4.pic,Margaret holds a BA in English literature from...
4,5,Buchanan,Steven,1955-03-04,EmpID5.pic,Steven Buchanan graduated from St. Andrews Uni...


In [15]:
orders.head()

,OrderID,CustomerID,EmployeeID,OrderDate,ShipperID
0,10248,90,5,1996-07-04,3
1,10249,81,6,1996-07-05,1
2,10250,34,4,1996-07-08,2
3,10251,84,3,1996-07-08,1
4,10252,76,4,1996-07-09,2


In [16]:
# Convert the string to a datetime object
orders['OrderDate'] = pd.to_datetime(orders['OrderDate'])
employees['BirthDate'] = pd.to_datetime(employees['BirthDate'])

In [17]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196 entries, 0 to 195
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   OrderID     196 non-null    int64         
 1   CustomerID  196 non-null    int64         
 2   EmployeeID  196 non-null    int64         
 3   OrderDate   196 non-null    datetime64[ns]
 4   ShipperID   196 non-null    int64         
dtypes: datetime64[ns](1), int64(4)
memory usage: 7.8 KB
